# Module 4 Homework

## More Walmart Cleaning

The file **Walmart_United_States\_&\_Canada.csv** contains data on all
Walmarts, Sam's Clubs and Murphy USA gas/diesel in the USA and Canada.
Note that Gas/Diesel and No Over Night Parking (NOP) are indicated if
known. These data can be obtained from the site
<http://www.poi-factory.com/node/25560>.


**Before you start.** You started cleaning this data set in Activity 4.1.  Start by copying over your code and fixing the encoding issue.

In [10]:
import pandas as pd
from dfply import *
from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter
import regex

In [11]:
# Copy your code from Activity 4.1 here

walmart = pd.read_csv("/home/fahad/module-4-homework-nameer1811/data/Walmart_United_States_&_Canada.csv",
                     names = ["Lat", "Long", "Store", "Address_and_Phone"]
                     )
walmart

,Lat,Long,Store,Address_and_Phone
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-..."
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,..."
...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter; #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230..."
6812,-108.379227,43.042858,"Walmart Supercenter; #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,(307) ..."
6813,-109.251020,41.579761,"Walmart Supercenter; #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,W..."
6814,-106.940967,44.779474,"Walmart Supercenter; #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828..."


In [12]:
(walmart
    >> select(X.Store)
    >> filter_by(~text_filter(X.Store, r"Walmart Supercent[re|er]", regex=True))
    >> filter_by(~text_filter(X.Store, r"Sam's Club|Murphy|Wm", regex=True))
    >> filter_by(~text_filter(X.Store, r"Walmart;\s#\d{1,4}", regex=True))
    >> filter_by(~text_filter(X.Store, r"Walmart [Pickup | Fuel]", regex=True))
    >> filter_by(~text_filter(X.Store, "; Supercenter"))
)

,Store


In [13]:
walmart_cleaned = (walmart
    >> mutate(Store = ifelse(X.Store.str.contains("; Supercenter"),X.Store.str.replace(";", ""), X.Store))
    >> mutate(Store = X.Store.str.replace(';', ','))
    >> mutate(Store_Name = X.Store.str.split(',').str.get(0),
             Store_number = X.Store.str.extract(r".*#([0-9]+)"),
             Gas_or_Diesel = X.Store.str.extract(r"(Gas.+)").replace(",","", regex=True))
    >>  mutate(Gas_or_Diesel = ifelse(X.Gas_or_Diesel.isna(), "Neither", X.Gas_or_Diesel))
)
walmart_cleaned

,Lat,Long,Store,Address_and_Phone,Store_Name,Store_number,Gas_or_Diesel
0,-114.005671,51.262567,"Walmart Supercentre, #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",Walmart Supercentre,1050,Neither
1,-111.900542,50.577939,"Walmart Supercentre, #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",Walmart Supercentre,3658,Neither
2,-114.039133,51.107253,"Walmart Supercentre, #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40...",Walmart Supercentre,3013,Neither
3,-114.138488,51.040871,"Walmart Supercentre, #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ...",Walmart Supercentre,3009,Gas
4,-114.028603,50.930551,"Walmart, #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",Walmart,1144,Neither
...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter, #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",Walmart Supercenter,4471,Gas
6812,-108.379227,43.042858,"Walmart Supercenter, #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,(307) ...",Walmart Supercenter,1457,Neither
6813,-109.251020,41.579761,"Walmart Supercenter, #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",Walmart Supercenter,1461,Neither
6814,-106.940967,44.779474,"Walmart Supercenter, #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",Walmart Supercenter,1508,Neither


1.  Some of the address columns contain `(NOP)` to indicated *No overnight parking*.  Extract this information into a new indicator column, then remove it from the address column.

In [14]:
# Your code here

walmart_address_nop = (walmart_cleaned 
>> mutate(No_Overnight_Parking = ifelse(X.Address_and_Phone.str.contains(r"\(NOP\)"), 1, 0))
>> mutate(Address_and_Phone = X.Address_and_Phone.str.replace(r"\(NOP\)", "", regex = True))
) 

walmart_address_nop

,Lat,Long,Store,Address_and_Phone,Store_Name,Store_number,Gas_or_Diesel,No_Overnight_Parking
0,-114.005671,51.262567,"Walmart Supercentre, #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",Walmart Supercentre,1050,Neither,0
1,-111.900542,50.577939,"Walmart Supercentre, #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",Walmart Supercentre,3658,Neither,0
2,-114.039133,51.107253,"Walmart Supercentre, #3013,","1110 57th Ave NE,Calgary ,,AB T2E 9B7,(403) 73...",Walmart Supercentre,3013,Neither,1
3,-114.138488,51.040871,"Walmart Supercentre, #3009,Gas,","1212 37 St SW,Calgary ,,AB T3C 1S3,(403) 242-2205",Walmart Supercentre,3009,Gas,1
4,-114.028603,50.930551,"Walmart, #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",Walmart,1144,Neither,0
...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter, #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",Walmart Supercenter,4471,Gas,0
6812,-108.379227,43.042858,"Walmart Supercenter, #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,(307) ...",Walmart Supercenter,1457,Neither,0
6813,-109.251020,41.579761,"Walmart Supercenter, #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",Walmart Supercenter,1461,Neither,0
6814,-106.940967,44.779474,"Walmart Supercenter, #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",Walmart Supercenter,1508,Neither,1


2.  The address column contains the phone number of most of the stores.  Extract this information into a new column.  There are some issue with different patterns, so use the divide-and-conquer approach is advised.

In [15]:
pd.set_option("display.max_colwidth", None)

In [16]:
# Your code here
#View Cell

# Tried multiple view and then condensed it to one single regex to make the code simpler.

# I think these are wrong phone numbers or phone number not available, I think we should just say no phone number available.

(
    walmart_address_nop
    >> select(X.Address_and_Phone)
    >> filter_by(~text_filter(X.Address_and_Phone,r"\(\d{3,4}_?\s?\)?\(?\s?-?\d{3}-?\s?\d{4}", regex=True))
)

,Address_and_Phone
354,"8303 Rogers Ave,Fort Smith,AR,72903 ,,(479) 452-161"
6135,"6040 Coit Rd,Plano,TX,75023,"
6193,"3440 S Bryant Blvd,San Angelo,TX,76903 ,,(325) 26-6599"


In [17]:
#Transform Cell

#All the 4 digits had 0, e.g. (3030), hence I thought maybe it is a ) mistakenly typed 0 since 0 and ) is mapped to the same button.

walmart_phone_work= (
    walmart_address_nop
    >> mutate(Phone_Number = X.Address_and_Phone.str.extract(r"(\(\d{3,4}_?\s?\)?\(?\s?-?\d{3}-?\s?\d{4})")
                                                    .replace(r"\((\d{3,4})_?\s?\)?\(?\s?-?(\d{3})-?\s?(\d{4})",r"(\1) \2-\3",regex=True))
    >> mutate(Phone_Number = X.Phone_Number.str.replace(r"\((\d\d\d)\d\)",r"(\1)", regex=True))
    >> mutate(Phone_Number = ifelse(X.Phone_Number.isna(), "No Phone Number Available", X.Phone_Number))
    >> mutate(Address = X.Address_and_Phone.str.replace(r",?\s?\(\d{3,4}_?\s?\)?\(?\s?-?\d{2,3}-?\s?\d{3,4}", "", regex=True))
    >> mutate(Address = X.Address.str.replace(r"\s?,,?", ", ", regex=True).str.replace(r"(\d),", r"\1", regex=True))
    >> drop(X.Address_and_Phone)
)

walmart_phone_work

,Lat,Long,Store,Store_Name,Store_number,Gas_or_Diesel,No_Overnight_Parking,Phone_Number,Address
0,-114.005671,51.262567,"Walmart Supercentre, #1050,",Walmart Supercentre,1050,Neither,0,(403) 945-1295,"2881 Main St SW, Airdrie, AB T4B 3G5"
1,-111.900542,50.577939,"Walmart Supercentre, #3658,",Walmart Supercentre,3658,Neither,0,(403) 793-2111,"917 3rd St W, Brooks, AB T1R 1L5"
2,-114.039133,51.107253,"Walmart Supercentre, #3013,",Walmart Supercentre,3013,Neither,1,(403) 730-0990,"1110 57th Ave NE, Calgary, AB T2E 9B7"
3,-114.138488,51.040871,"Walmart Supercentre, #3009,Gas,",Walmart Supercentre,3009,Gas,1,(403) 242-2205,"1212 37 St SW, Calgary, AB T3C 1S3"
4,-114.028603,50.930551,"Walmart, #1144,",Walmart,1144,Neither,0,(403) 225-6638,"1221 Canyon Meadows Dr SE, Calgary, AB T2J 6G2"
...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter, #4471,Gas,",Walmart Supercenter,4471,Gas,0,(307) 417-3001,"2390 E Cedar St; I-80 Exit 214 Rawlins, WY, 82301"
6812,-108.379227,43.042858,"Walmart Supercenter, #1457,",Walmart Supercenter,1457,Neither,0,(307) 856-3261,"1733 N Federal Blvd, Riverton, WY, 82501"
6813,-109.251020,41.579761,"Walmart Supercenter, #1461,",Walmart Supercenter,1461,Neither,0,(307) 362-1957,"201 Gateway Blvd; I-80 Exit 102 Rock Springs, WY, 82901"
6814,-106.940967,44.779474,"Walmart Supercenter, #1508,",Walmart Supercenter,1508,Neither,1,(307) 674-6492,"1695 Coffeen Ave; I-90 Exit 25 Sheridan, WY, 82801"


3.  Extract the country from the address column.

In [18]:
# Your code here
#View Cell
(
    walmart_phone_work
    >> filter_by(~text_filter(X.Address,r", [A-Z]{2},?\s?\d{4,5}?", regex=True))  #Vermont is missing 0 before their zipcode so repetition was 4 or 5 for normal zip
    >> filter_by(~text_filter(X.Address,r", [A-Z]{2}\s[A-Z][0-9][A-Z]\s[a-zA-Z0-9]{3}", regex=True))
)

,Lat,Long,Store,Store_Name,Store_number,Gas_or_Diesel,No_Overnight_Parking,Phone_Number,Address
42,-114.141037,51.799211,"Walmart Supercentre, #1084,",Walmart Supercentre,1084,Neither,0,(403) 556-3844,"6900 46th St, Olds OA2 AB T4H"
5085,-73.103305,46.023207,"Walmart, #1174,",Walmart,1174,Neither,0,(450) 743-2992,"450 Boul Poliquin, Sorel-Tracy, QC., J3P 7R5"


In [19]:
#Transform Cell
walmart_countries = (
    walmart_phone_work
    >> mutate(Country = ifelse(X.Address.str.contains(r", [A-Z]{2},\s\d{4,5}"), "USA", "Canada"))
)

walmart_countries

,Lat,Long,Store,Store_Name,Store_number,Gas_or_Diesel,No_Overnight_Parking,Phone_Number,Address,Country
0,-114.005671,51.262567,"Walmart Supercentre, #1050,",Walmart Supercentre,1050,Neither,0,(403) 945-1295,"2881 Main St SW, Airdrie, AB T4B 3G5",Canada
1,-111.900542,50.577939,"Walmart Supercentre, #3658,",Walmart Supercentre,3658,Neither,0,(403) 793-2111,"917 3rd St W, Brooks, AB T1R 1L5",Canada
2,-114.039133,51.107253,"Walmart Supercentre, #3013,",Walmart Supercentre,3013,Neither,1,(403) 730-0990,"1110 57th Ave NE, Calgary, AB T2E 9B7",Canada
3,-114.138488,51.040871,"Walmart Supercentre, #3009,Gas,",Walmart Supercentre,3009,Gas,1,(403) 242-2205,"1212 37 St SW, Calgary, AB T3C 1S3",Canada
4,-114.028603,50.930551,"Walmart, #1144,",Walmart,1144,Neither,0,(403) 225-6638,"1221 Canyon Meadows Dr SE, Calgary, AB T2J 6G2",Canada
...,...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter, #4471,Gas,",Walmart Supercenter,4471,Gas,0,(307) 417-3001,"2390 E Cedar St; I-80 Exit 214 Rawlins, WY, 82301",USA
6812,-108.379227,43.042858,"Walmart Supercenter, #1457,",Walmart Supercenter,1457,Neither,0,(307) 856-3261,"1733 N Federal Blvd, Riverton, WY, 82501",USA
6813,-109.251020,41.579761,"Walmart Supercenter, #1461,",Walmart Supercenter,1461,Neither,0,(307) 362-1957,"201 Gateway Blvd; I-80 Exit 102 Rock Springs, WY, 82901",USA
6814,-106.940967,44.779474,"Walmart Supercenter, #1508,",Walmart Supercenter,1508,Neither,1,(307) 674-6492,"1695 Coffeen Ave; I-90 Exit 25 Sheridan, WY, 82801",USA


4.  Extract the state or province from the address columns

In [20]:
# Your code here
walmart_states = (
    walmart_countries
    >> mutate(Province_or_State = X.Address.str.replace(r".*, ([A-Z]{2}),?\s?[A-Z]?[0-9]?[A-Z]?\s?[a-zA-Z0-9]{3}?.*", r"\1",regex=True))
    >> mutate(Province_or_State = case_when((X.Province_or_State.str.contains("QC"), "QC"),
                                            (X.Province_or_State.str.contains("OA2 AB T4H"), "AB"),
                                            (True, X.Province_or_State)))
)

walmart_states.Province_or_State.unique()

array(['AB', 'AK', 'AL', 'AR', 'AZ', 'BC', 'CA', 'CO', 'CT', 'DC', 'DE',
       'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA',
       'MB', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NB', 'NC', 'ND',
       'NE', 'NH', 'NJ', 'NL', 'NM', 'NS', 'NT', 'NV', 'NY', 'OH', 'OK',
       'ON', 'OR', 'PA', 'PE', 'QC', 'RI', 'SC', 'SD', 'SK', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'YT'], dtype=object)

5. Combine all of your transformations into one pipe, then re-factor your code to be more readable.

In [21]:
# One pipe for all transformations.
(
    walmart
    >> mutate(Store = ifelse(X.Store.str.contains("; Supercenter"),X.Store.str.replace(";", ""), X.Store))
    >> mutate(Store = X.Store.str.replace(';', ','))
    >> mutate(Store_Name = X.Store.str.split(',').str.get(0),
             Store_number = X.Store.str.extract(r".*#([0-9]+)"),
             Gas_or_Diesel = X.Store.str.extract(r"(Gas.+)").replace(",","", regex=True))
    >> mutate(Gas_or_Diesel = ifelse(X.Gas_or_Diesel.isna(), "Neither", X.Gas_or_Diesel))
    >> mutate(No_Overnight_Parking = ifelse(X.Address_and_Phone.str.contains( r"\(NOP\)"), 1, 0))
    >> mutate(Address_and_Phone = X.Address_and_Phone.str.replace(r"\(NOP\)", "", regex = True))
    >> mutate(Phone_Number = X.Address_and_Phone.str.extract(r"(\(\d{3,4}_?\s?\)?\(?\s?-?\d{3}-?\s?\d{4})")
                                                    .replace(r"\((\d{3,4})_?\s?\)?\(?\s?-?(\d{3})-?\s?(\d{4})",r"(\1) \2-\3",regex=True)
                                                    .replace(r"\((\d\d\d)\d\)",r"(\1)", regex=True))
    >> mutate(Phone_Number = ifelse(X.Phone_Number.isna(), "No Phone Number Available", X.Phone_Number))
    >> mutate(Address = X.Address_and_Phone.str.replace(r",?\s?\(\d{3,4}_?\s?\)?\(?\s?-?\d{2,3}-?\s?\d{3,4}", "", regex=True)
                                               .replace(r"\s?,,?", ", ", regex=True).str.replace(r"(\d),", r"\1", regex=True))
    >> mutate(Country = ifelse(X.Address.str.contains( r", [A-Z]{2},\s\d{4,5}"), "USA", "Canada"))
    >> mutate(Province_or_State = X.Address.str.replace(r".*, ([A-Z]{2}),?\s?[A-Z]?[0-9]?[A-Z]?\s?[a-zA-Z0-9]{3}?.*", r"\1",regex=True))
    >> mutate(Province_or_State = case_when((X.Province_or_State.str.contains("QC"), "QC"),
                                            (X.Province_or_State.str.contains("OA2 AB T4H"), "AB"),
                                            (True, X.Province_or_State)))
    >> drop(X.Address_and_Phone, X.Store)
)

,Lat,Long,Store_Name,Store_number,Gas_or_Diesel,No_Overnight_Parking,Phone_Number,Address,Country,Province_or_State
0,-114.005671,51.262567,Walmart Supercentre,1050,Neither,0,(403) 945-1295,"2881 Main St SW, Airdrie, AB T4B 3G5",Canada,AB
1,-111.900542,50.577939,Walmart Supercentre,3658,Neither,0,(403) 793-2111,"917 3rd St W, Brooks, AB T1R 1L5",Canada,AB
2,-114.039133,51.107253,Walmart Supercentre,3013,Neither,1,(403) 730-0990,"1110 57th Ave NE, Calgary, AB T2E 9B7",Canada,AB
3,-114.138488,51.040871,Walmart Supercentre,3009,Gas,1,(403) 242-2205,"1212 37 St SW, Calgary, AB T3C 1S3",Canada,AB
4,-114.028603,50.930551,Walmart,1144,Neither,0,(403) 225-6638,"1221 Canyon Meadows Dr SE, Calgary, AB T2J 6G2",Canada,AB
...,...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,Walmart Supercenter,4471,Gas,0,(307) 417-3001,"2390 E Cedar St; I-80 Exit 214 Rawlins, WY, 82301",USA,WY
6812,-108.379227,43.042858,Walmart Supercenter,1457,Neither,0,(307) 856-3261,"1733 N Federal Blvd, Riverton, WY, 82501",USA,WY
6813,-109.251020,41.579761,Walmart Supercenter,1461,Neither,0,(307) 362-1957,"201 Gateway Blvd; I-80 Exit 102 Rock Springs, WY, 82901",USA,WY
6814,-106.940967,44.779474,Walmart Supercenter,1508,Neither,1,(307) 674-6492,"1695 Coffeen Ave; I-90 Exit 25 Sheridan, WY, 82801",USA,WY


In [22]:
# Refactored coded here (definitions/lambdas then the pipe)

#Filters 
filter_supercenter = X.Store.str.contains("; Supercenter")
filter_NOP = X.Address_and_Phone.str.contains(r"\(NOP\)")
filter_USA = X.Address.str.contains( r", [A-Z]{2},\s\d{4,5}")
filter_Quebec = X.Province_or_State.str.contains("QC")
filter_AB = X.Province_or_State.str.contains("OA2 AB T4H")


#Transforms

get_store_name = X.Store.str.split(',').str.get(0)

extract_store_number = X.Store.str.extract(r".*#([0-9]+)")
extract_gas_info_and_format = X.Store.str.extract(r"(Gas.+)").replace(",","", regex=True)
extract_phone_number_and_format = (X.Address_and_Phone.str.extract(r"(\(\d{3,4}_?\s?\)?\(?\s?-?\d{3}-?\s?\d{4})")
                                                    .replace(r"\((\d{3,4})_?\s?\)?\(?\s?-?(\d{3})-?\s?(\d{4})",r"(\1) \2-\3",regex=True)
                                                    .replace(r"\((\d\d\d)\d\)",r"(\1)", regex=True))

replace_supercenter_semicolon = X.Store.str.replace(";", "")
change_colon_to_comma = X.Store.str.replace(';', ',')
remove_NOP = X.Address_and_Phone.str.replace(r"\(NOP\)", "", regex = True)
extract_address_and_format = (X.Address_and_Phone.str.replace(r",?\s?\(\d{3,4}_?\s?\)?\(?\s?-?\d{2,3}-?\s?\d{3,4}", "", regex=True)
                                               .replace(r"\s?,,?", ", ", regex=True).str.replace(r"(\d),", r"\1", regex=True))
get_province_or_state = X.Address.str.replace(r".*, ([A-Z]{2}),?\s?[A-Z]?[0-9]?[A-Z]?\s?[a-zA-Z0-9]{3}?.*", r"\1",regex=True)

no_phone_number_check = X.Phone_Number.isna()

no_phone_available = "No Phone Number Available"
USA = "USA"
Canada = "Canada"

leave_store_unchanged = X.Store
leave_phone_number_unchanged = X.Phone_Number
leave_province_or_state_unchanged = X.Province_or_State
leave_gas_or_diesel_unchanged = X.Gas_or_Diesel

change_super_center_semicolon = ifelse(filter_supercenter,replace_supercenter_semicolon, leave_store_unchanged)
gas_or_diesel_or_neither = ifelse(X.Gas_or_Diesel.isna(), "Neither", leave_gas_or_diesel_unchanged)
no_over_night_parking_indicator = ifelse(filter_NOP, 1, 0)
no_phone_available_marker = ifelse(no_phone_number_check, no_phone_available, leave_phone_number_unchanged)
USA_Canada_indicator = ifelse(filter_USA, USA, Canada)
cases_for_state_province = case_when((filter_Quebec, "QC"),(filter_AB, "AB"),(True, leave_province_or_state_unchanged))


In [23]:
walmart_finished = (
    walmart
    >> mutate(Store = change_super_center_semicolon)
    >> mutate(Store = change_colon_to_comma)
    >> mutate(Store_Name = get_store_name, Store_number = extract_store_number, Gas_or_Diesel = extract_gas_info_and_format)
    >> mutate(Gas_or_Diesel = gas_or_diesel_or_neither)
    >> mutate(No_Overnight_Parking = no_over_night_parking_indicator)
    >> mutate(Address_and_Phone = remove_NOP)
    >> mutate(Phone_Number = extract_phone_number_and_format)
    >> mutate(Phone_Number = no_phone_available_marker)
    >> mutate(Address = extract_address_and_format)
    >> mutate(Country = USA_Canada_indicator)
    >> mutate(Province_or_State = get_province_or_state)
    >> mutate(Province_or_State = cases_for_state_province)
    >> drop(X.Address_and_Phone, X.Store)
)

walmart_finished

,Lat,Long,Store_Name,Store_number,Gas_or_Diesel,No_Overnight_Parking,Phone_Number,Address,Country,Province_or_State
0,-114.005671,51.262567,Walmart Supercentre,1050,Neither,0,(403) 945-1295,"2881 Main St SW, Airdrie, AB T4B 3G5",Canada,AB
1,-111.900542,50.577939,Walmart Supercentre,3658,Neither,0,(403) 793-2111,"917 3rd St W, Brooks, AB T1R 1L5",Canada,AB
2,-114.039133,51.107253,Walmart Supercentre,3013,Neither,1,(403) 730-0990,"1110 57th Ave NE, Calgary, AB T2E 9B7",Canada,AB
3,-114.138488,51.040871,Walmart Supercentre,3009,Gas,1,(403) 242-2205,"1212 37 St SW, Calgary, AB T3C 1S3",Canada,AB
4,-114.028603,50.930551,Walmart,1144,Neither,0,(403) 225-6638,"1221 Canyon Meadows Dr SE, Calgary, AB T2J 6G2",Canada,AB
...,...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,Walmart Supercenter,4471,Gas,0,(307) 417-3001,"2390 E Cedar St; I-80 Exit 214 Rawlins, WY, 82301",USA,WY
6812,-108.379227,43.042858,Walmart Supercenter,1457,Neither,0,(307) 856-3261,"1733 N Federal Blvd, Riverton, WY, 82501",USA,WY
6813,-109.251020,41.579761,Walmart Supercenter,1461,Neither,0,(307) 362-1957,"201 Gateway Blvd; I-80 Exit 102 Rock Springs, WY, 82901",USA,WY
6814,-106.940967,44.779474,Walmart Supercenter,1508,Neither,1,(307) 674-6492,"1695 Coffeen Ave; I-90 Exit 25 Sheridan, WY, 82801",USA,WY


In [24]:
# Write the results to a file named walmart_locations_clean.csv.  Make sure to include this file in your submission on D2L

walmart_finished.to_csv("walmart_locations_clean.csv")